In [1]:
import tensorflow_addons as tfa
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow
import numpy as np
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense,
    Flatten,
    Dropout,
    BatchNormalization
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber
from sklearn.preprocessing import StandardScaler

In [2]:
# coefficient of determination (R^2) for regression
def r_square(y_true, y_pred):
    from tensorflow.keras import backend as K
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return (1 - SS_res/(SS_tot + K.epsilon()))

def r_square_loss(y_true, y_pred):
    from tensorflow.keras import backend as K
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return 1 - ( 1 - SS_res/(SS_tot + K.epsilon()))

In [3]:
# load data
x_train = pd.read_csv("./dat300-h2020-ca1/X_train.csv")
x_test = pd.read_csv("./dat300-h2020-ca1/X_test.csv")
y_train = pd.read_csv("./dat300-h2020-ca1/y_train.csv")

In [ ]:
# show columns with lots of zeros
#x_train = x_train.replace(0.0, np.nan)
#missing_values = x_train.isnull().sum()

In [ ]:
# drop columns with lots of zeros
#x_train = x_train.drop(columns=["Blast Furnace Slag (component 2)(kg in a m^3 mixture)",
#                               "Fly Ash (component 3)(kg in a m^3 mixture)",
#                               "Superplasticizer (component 5)(kg in a m^3 mixture)"])

#x_test = x_test.drop(columns=["Blast Furnace Slag (component 2)(kg in a m^3 mixture)",
#                               "Fly Ash (component 3)(kg in a m^3 mixture)",
#                               "Superplasticizer (component 5)(kg in a m^3 mixture)"])

In [4]:
# remove outliers
print(x_train.shape)
Q1 = x_train.quantile(0.25)
Q3 = x_train.quantile(0.75)
IQR = Q3-Q1
mask = ((x_train < (Q1 - 1.5 * IQR)) | (x_train > (Q3 + 1.5 * IQR))).any(axis=1)
print("Observations BEFORE removing outliers:", x_train.shape)
x_train = x_train[~mask]
print(x_train.shape)
y_train = y_train[~mask]
# X = X.values
# _test = x_test.values
print("Observations AFTER removing outliers:", x_train.shape)

(618, 8)
Observations BEFORE removing outliers: (618, 8)
(569, 8)
Observations AFTER removing outliers: (569, 8)


In [ ]:
# BOXPLOT
a = x_train.plot(kind="box", subplots=True, figsize=(12, 12), layout=(5,4), sharex=False, sharey=False)

In [5]:
# StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [6]:
# keras ANN
#model.add(BatchNormalization())
sample_vector_length = x_train.shape[1]
model = Sequential([ 
                   Dense(200, activation="relu", input_shape=(sample_vector_length,)),
                   Dense(100, activation="relu"),
                   Dense(100, activation="relu"),
                   Dense(1)])
model.compile(optimizer=tensorflow.keras.optimizers.Nadam(),
             loss=,
             metrics=[r_square])

In [7]:
history = model.fit(x_train, y_train, epochs=300, validation_split=0.2, batch_size=25, verbose=1)

Epoch 1/300
19/19 [==============================] - 0s 11ms/step - loss: 5.4275 - r_square: -4.5045 - val_loss: 4.7849 - val_r_square: -3.9595
Epoch 2/300
19/19 [==============================] - 0s 4ms/step - loss: 2.3667 - r_square: -1.2888 - val_loss: 0.8229 - val_r_square: 0.1895
Epoch 3/300
19/19 [==============================] - 0s 4ms/step - loss: 0.6604 - r_square: 0.3411 - val_loss: 0.6398 - val_r_square: 0.3935
Epoch 4/300
19/19 [==============================] - 0s 4ms/step - loss: 0.5757 - r_square: 0.4133 - val_loss: 0.6240 - val_r_square: 0.4095
Epoch 5/300
19/19 [==============================] - 0s 4ms/step - loss: 0.5023 - r_square: 0.4996 - val_loss: 0.5883 - val_r_square: 0.4374
Epoch 6/300
19/19 [==============================] - 0s 4ms/step - loss: 0.4795 - r_square: 0.5241 - val_loss: 0.5694 - val_r_square: 0.4584
Epoch 7/300
19/19 [==============================] - 0s 4ms/step - loss: 0.5015 - r_square: 0.4287 - val_loss: 0.7113 - val_r_square: 0.2968
Epoch 8/3

In [ ]:
# plot mse metric
epochs_from_end = 100
epoch_x_values = list(range(len(history.history['loss'])-epochs_from_end, len(history.history['loss'])))
plt.plot(epoch_x_values, history.history['loss'][-epochs_from_end:])
plt.plot(epoch_x_values, history.history['val_loss'][-epochs_from_end:])
plt.title('model mse')
plt.ylabel('mse')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

In [ ]:
# plot r^2 metric
epochs_from_end = 100
epoch_x_values = list(range(len(history.history['r_square'])-epochs_from_end, len(history.history['r_square'])))
plt.plot(epoch_x_values, history.history['r_square'][-epochs_from_end:])
plt.plot(epoch_x_values, history.history['val_r_square'][-epochs_from_end:])
plt.title('model r_square')
plt.ylabel('r_square')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='lower right')
plt.show()

In [ ]:
# show some info about model and training to save for later reference
print("StandardScaler\n")
print(model.summary(), "\n")
print(history.params, "\n")
print(f"mean train loss last 10 epochs: {np.mean(history.history['loss'][-10:])}")
print(f"mean val loss last 10 epochs: {np.mean(history.history['val_loss'][-10:])}")
print(f"mean train r^2 last 10 epochs: {np.mean(history.history['r_square'][-10:])}")
print(f"mean val r^2 last 10 epochs: {np.mean(history.history['val_r_square'][-10:])}")

In [8]:
# train new model on all traindata
pred_model = Sequential([ 
                   Dense(200, activation="relu", input_shape=(sample_vector_length,)),
                   Dense(100, activation="relu"),
                   Dense(100, activation="relu"),
                   Dense(1)])
pred_model.compile(optimizer=tensorflow.keras.optimizers.Nadam(),
             loss=r_square_loss,
             metrics=[r_square])

In [9]:
pred_history = pred_model.fit(x_train, y_train, epochs=300, batch_size=25, verbose=1)

Epoch 1/300
23/23 [==============================] - 0s 3ms/step - loss: 5.4158 - r_square: -4.4021
Epoch 2/300
23/23 [==============================] - 0s 3ms/step - loss: 1.3763 - r_square: -0.3684
Epoch 3/300
23/23 [==============================] - 0s 3ms/step - loss: 0.5727 - r_square: 0.4292
Epoch 4/300
23/23 [==============================] - 0s 3ms/step - loss: 0.4981 - r_square: 0.5026
Epoch 5/300
23/23 [==============================] - 0s 3ms/step - loss: 0.5119 - r_square: 0.4899
Epoch 6/300
23/23 [==============================] - 0s 3ms/step - loss: 0.4643 - r_square: 0.5373
Epoch 7/300
23/23 [==============================] - 0s 3ms/step - loss: 0.4455 - r_square: 0.5540
Epoch 8/300
23/23 [==============================] - 0s 3ms/step - loss: 0.4082 - r_square: 0.5911
Epoch 9/300
23/23 [==============================] - 0s 3ms/step - loss: 0.4120 - r_square: 0.5828
Epoch 10/300
23/23 [==============================] - 0s 3ms/step - loss: 0.3649 - r_square: 0.6361
Epoch 1

In [10]:
# make prediction on test data
y_pred = pred_model.predict(x_test)

In [11]:
# save prediction to submission format
dfscore = pd.DataFrame(y_pred)
ID = np.arange(0, len(dfscore), dtype=np.int64)
dfID = pd.DataFrame(ID)
dfscore.columns = ["Predicted"]
dfID.columns = ["Id"]
final_df = pd.concat([dfID, dfscore], axis=1)
final_df.to_csv("Julius&Markus_comp1_ver1.10.csv", sep=",", index=False)

In [12]:
# show some info about model and training to save for later reference
print("standardscaler\n")
print(pred_model.summary(), "\n")
print(pred_history.params, "\n")
print(f"mean train loss last 10 epochs: {np.mean(history.history['loss'][-10:])}")
print(f"mean train r^2 last 10 epochs: {np.mean(pred_history.history['r_square'][-10:])}")

standardscaler

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 200)               1800      
_________________________________________________________________
dense_5 (Dense)              (None, 100)               20100     
_________________________________________________________________
dense_6 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 101       
Total params: 32,101
Trainable params: 32,101
Non-trainable params: 0
_________________________________________________________________
None 

{'verbose': 1, 'epochs': 300, 'steps': 23} 

mean train loss last 10 epochs: 0.037236590310931204
mean train r^2 last 10 epochs: 0.9748851358890533
